In [1]:
import datetime
import time
import json
# import psycopg2

# url = "hdfs://127.0.0.1:9000/socmed_input/"
# out_url = "hdfs://127.0.0.1:9000/socmed_output/"

# conn = psycopg2.connect(database="bigdatatest",
#     host="localhost",
#     user="geraldakbar",
#     password='bigdata'
# )
# cur = conn.cursor()

In [2]:
def normalizeTime(time):
    res = ''
    for i in range(len(time)):
        if i == 6 or i == 7:
            res += '0'
        elif i == 4:
            if int(time[i]) < 5:
                res += '0'
            else:
                res += '5'
        else:
            res += time[i] 
    return res

def parseTime(type, time):
    if type == "twitter":
        ts = t.strftime('%Y-%m-%d %H:%M:%S', t.strptime(time,'%a %b %d %H:%M:%S +0000 %Y'))
        return ts[:10]
    elif type == "instagram" or type == "other":
        time = str(datetime.datetime.fromtimestamp(int(time)).date())
        return time[:10]
    else:
        return time[:10] + ' ' + normalizeTime(time[11:19])

In [3]:
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

ssc = StreamingContext(sc,1) #stream each one second
ssc.checkpoint("./checkpoint")
lines = KafkaUtils.createDirectStream(ssc, ['instagram'],
                                      {"metadata.broker.list": "localhost:9092"})
def getIG(lines):
    # map the lines to a key-value pair
    def map(line):
        date = line['created_time']
        # format date
        date = str(datetime.datetime.fromtimestamp(int(date)))
        date = date[:10] + ' ' + normalizeTime(date[11:19])
        return (("instagram", date), 1)
    def userMap(line):
        user = line['user']['username']
        date = line['created_time']
        # format date
        date = str(datetime.datetime.fromtimestamp(int(date)))
        date = date[:10] + ' ' + normalizeTime(date[11:19])
        return (("instagram", date, user), 1)

    lines = lines.window(5, 5)
    lines = lines.map(lambda x: json.loads(x[1]))
    igPost = lines.map(map)
    igPost = igPost.reduceByKey(lambda x, y: x + y)

    user = lines.map(userMap)
    user = user.transform(lambda rdd: rdd.distinct())
    user = user.map(lambda x: ((x[0][0], x[0][1]), x[1]))
    user = user.reduceByKey(lambda x, y: x + y)

    igPost = igPost.join(user).map(lambda x: (x[0], x[1][0], x[1][1]))
    igPost = igPost.map(lambda x: (x[0][0], x[0][1], x[1], x[2]))
    #igPost = masuk ke postgre
    return igPost

result = getIG(lines)
# Print
result.pprint()
ssc.start()
ssc.awaitTermination()

# test = igPost.take(10)
# insertToSQL(test,conn,cur)

-------------------------------------------
Time: 2023-04-17 23:40:37
-------------------------------------------
('instagram', '2023-04-05 00:10:00', 1, 1)
('instagram', '2023-04-05 00:05:00', 1, 1)

-------------------------------------------
Time: 2023-04-17 23:40:42
-------------------------------------------
('instagram', '2023-04-05 00:10:00', 1, 1)

-------------------------------------------
Time: 2023-04-17 23:40:47
-------------------------------------------
('instagram', '2023-04-05 00:10:00', 1, 1)

-------------------------------------------
Time: 2023-04-17 23:40:52
-------------------------------------------
('instagram', '2023-04-05 00:10:00', 1, 1)

-------------------------------------------
Time: 2023-04-17 23:40:57
-------------------------------------------
('instagram', '2023-04-05 00:10:00', 2, 1)
('instagram', '2023-04-04 23:40:00', 1, 1)
('instagram', '2023-04-04 23:30:00', 1, 1)

-------------------------------------------
Time: 2023-04-17 23:41:02
-----------

KeyboardInterrupt: 

In [4]:
def insertToSQL(data,connection,cursor):
    for element in data:
        social_media = element[0]
        timestamp = element[1]
        count = element[2]
        uniquecount = element[3]

        query = f"""INSERT INTO social_media_data (social_media, timestamp, count, uniquecount)
                    VALUES ('{social_media}', '{timestamp}', {count}, {uniquecount})
                    ON CONFLICT (social_media, timestamp) DO UPDATE
                    SET count = CASE WHEN social_media_data.count IS NULL THEN 1 ELSE social_media_data.count + {count} END,
                    uniquecount = CASE WHEN social_media_data.uniquecount IS NULL THEN 1 ELSE social_media_data.uniquecount + {uniquecount} END,
                    updated_at = NOW();"""



        cursor.execute(query)

    connection.commit()